# Convergence of the exact time propagation

The goal of this notebook is to test the convergence of the exact time propagation with respect to the densification of the continuum states spectrum and the range of their wavenumbers (or energies). This allows to get a grasp on the default values you would use, depending on the desired accuracy. 

The main message of this notebook is the following: to make sure that your exact time propagation calculation is converged with respect to the continuum basis set you must follow the following steps:

* Firstly, make sure that the completeness relation using your basis set is verified for the initial wavepacket. To that end, it is advised to use the smallest $k_{max}$ possible, so that the exact time propagation for $t = 0$ conveniently reproduces the initial wavepacket. At that point, is may not be interesting to use too small $h_k$ values.

* Lastly, once you found a suiting value for $k_{max}$, make a convergence study with respect to $h_k$. To that end, you have to make sure that decreasing $h_k$ does not change the time propagation of the wavepacket for the maximal time you desire.

For the second point, remember that the RAGE theorem tells you that the continuum states contributions to the time propagation of the wavepacket vanishes in the long time limit: it might not be interesting to look at what is happening for too long times, since using only the bound states to propagate the wavepacket should suffice, while converging the exact propagation might become time-consuming because too small values for $h_k$ are required.

**Import useful modules and classes.**

In [ ]:
# Make the notebook aware of some of the SiegPy module classes
from siegpy import SWPBasisSet, Gaussian, SWPotential
# Other imports
import numpy as np
import matplotlib.pyplot as plt

## Define the potential

In [ ]:
l = 3.0
V0 = 8.0
xgrid = np.linspace(-l/2., l/2., 201)
pot = SWPotential(l, V0, grid=xgrid)

## Create multiple exact basis sets

An exact basis set is made of bound and continuum states. While the bound states form a discrete spectrum that is easy to find, there is a continuum of states of real, positive energies. This continuum has to be discretized, and we will study how the parameters affecting this continuum of states affect the precision of the time propagation of a wavepacket.

The parameters in question are the wavenumber grid step $h_k$ and the range of the grid $k_{max}$, *i.e.* the highest wavenumber of the state in the basis set. A set of exact basis sets is created to study the impact of both parameters.

To make the calculations quicker, we only add the even continuum states to the exact basis sets. This implies that, in order to get physically relevant results, an even test function will have to be used.

In [ ]:
# Find the bound states of the potential.
# They are found automatically, without specifying values for the 
# wavenumber grid of input guesses
siegerts = SWPBasisSet.find_Siegert_states(pot, 0, 1, 0, grid=xgrid)
bnds = siegerts.bounds

In [ ]:
# Define multiple continuum states basis sets
conts = []
h_ks = [1, 0.5, 0.1, 0.05, 0.025, 0.01]
k_maxs = [5, 10, 15, 20, 25, 30, 50]
#h_ks = [1, 0.5, 0.1]
#k_maxs = [5, 10, 15, 20]
for h in h_ks:
    for k in k_maxs:
        print("h_k = {}, k_max = {}".format(h, k))
        # Use even_only=True to save computation time
        conts.append(SWPBasisSet.find_continuum_states(pot, k, h, even_only=True, grid=xgrid))

In [ ]:
# Define the exact basis sets made of bound and continuum states
exacts = [cont + bnds for cont in conts]

## Define the test function

We chose to add only the even continuum states to the exact basis sets, we therefore need to use a centered test function for the results to have a physical meaning (*e.g.* to neglect the odd states contributions to the time propagation).

In [ ]:
x_c = 0.0
sigma = l/20.
gauss = Gaussian(sigma, x_c, grid=xgrid)

## Compute the time propagation for different times

We finally compute the time propagation of the wavepacket using the different basis sets. To that end, we must define `time_grid` first.

In [ ]:
time_grid = [0.0, 0.25, 0.5, 1.0, 3.0]
exact_tps = [exact.exact_propagation(gauss, time_grid) for exact in exacts]

## Plot the convergence with respect to both parameters

It is now possible to check the influence of both parameters separately. One important point to look for is the influence of the time parameter also: does the precision also vary with the time at which the propagation is evaluated?

### Convergence with respect to $k_{max}$

This is probably the easiest parameter to predict: by computing the convergence of the completeness relation using the initial wavepacket as test function, it is easy to see for which $k_{max}$ it is sufficient for the completeness relation to be fulfilled.

In [ ]:
exacts[-1].plot_completeness_convergence(gauss, MLE=False)

This plot tells us that the time propagation must not change between two calculations using the same value for $h_k$ if $k_{max} \gtrsim 15$.

This is validated by the $t=0$ study, which is, by the way, just another way of experimenting the convergence of the exact completeness relation with respect to $k_{max}$, since the initial wavepacket is expected to be reproduced. 

This is what is presented below: once $k_{max}$ is high enough (here, above 20), the expression of the initial wavepacket is  converged **for a given h_k**. To reach a truly converged initial wavepacket, the wavenumber grid step must be decreased. For the lowest $h_k$, the $k_{max}$ does not change.

In [ ]:
i_t = 0 
t = time_grid[i_t]
for i_h, h in enumerate(h_ks):
    for i_k, k in enumerate(k_maxs):
        lab = "h_k = {}, k_max = {}".format(h, k)
        i_b = i_h*len(k_maxs)+i_k
        tp = np.abs(exact_tps[i_b][i_t])
        plt.plot(xgrid, tp, label=lab)
    plt.title("t = {}".format(t))
    plt.legend(loc=6, bbox_to_anchor=(1, 0.5))
    plt.show()

#### Absolute Errors 

What we just presented is exactly what was found while studying the convergence of the exact completeness relation. Another way to look at the precision of the results is to directly look for the absolute errors, since the reference is the initial wavepacket.

In [ ]:
i_t = 0 
t = time_grid[i_t]
ref = gauss.values
for i_h, h in enumerate(h_ks):
    for i_k, k in enumerate(k_maxs):
        lab = "h_k = {}, k_max = {}".format(h, k)
        i_b = i_h*len(k_maxs)+i_k
        tp = np.abs(exact_tps[i_b][i_t])
        plt.plot(xgrid, np.abs(tp-ref), label=lab)
    plt.yscale('log')
    plt.title("t = {}".format(t))
    plt.legend(loc=6, bbox_to_anchor=(1, 0.5))
    plt.show()

For the largest values of $h_k$, there is a convergence of the absolute errors: increasing $k_{max}$ does not affect the quality of the results. Nonetheless, the level of the absolute error is quite high, above $10^{-2}$. When $h_k$ is decreased, then the basis set is denser, and the absolute errors do not superimpose in the log scale. However, the level of the absolute error is below $10^{-2}$ for each $k_{max} \geq 15$ for the smallest $h_k$, and never reaches smaller values if $h_k$ is further decreased (for a given $k_{max}$). This means the convergence with respect to $h_k$ has been reached for this given $k_{max}$.

We know that once $k_{max}$ is high enough, there is a convergence of the wavepacket expansion. Is this also true for larger times? This what is tested below.

#### Convergence with respect ot $k_{max}$ for large time

Let us produce the same plots as earlier but for the maximal time:

In [ ]:
i_t = -1 
t = time_grid[i_t]
for i_h, h in enumerate(h_ks):
    for i_k, k in enumerate(k_maxs):
        lab = "h_k = {}, k_max = {}".format(h, k)
        i_b = i_h*len(k_maxs)+i_k
        tp = np.abs(exact_tps[i_b][i_t])
        plt.plot(xgrid, tp, label=lab)
    plt.title("t = {}".format(t))
    plt.legend(loc=6, bbox_to_anchor=(1, 0.5))
    plt.show()

Again, a convergence as $k_{max}$ increases is observed. This is particularly seen for the largest values of $h_k$, where it requires $k_{max} \geq 15$ to reach a convergence. 

However, the situation is not that clear for the lowest values of $h_k$: the propagated wavepacket obtained with $k_{max} = 5$ is already really close to the converged result. Why is that?

This is simply due to the RAGE theorem introduced in the `time_propagation` notebook: as long as the time increases, the continuum states contribution to the propagation of the wavepacket tend to zero. For $t = 3.0$, the continuum contribution to the time propagation is very small, and this explains the small importance of the $k_{max}$ value.

This means that once the $k_{max}$ value is high enough to reproduce the initial wavepacket, then it will not impact the time propagation of the wavepacket, even for large time. 

However, you may also note that the shape of the wavepacket at convergence (*i.e.* for the highest value of $k_{max}$) greatly depends on $h_k$. However, once $h_k$ is small enough (*e.g.* below 0.05), the shape of the converged wavepacket remains almost the same. This means that this parameter is very important. It will be the topic of the next section of the notebook.

#### Absolute Errors 

Before moving to the influence of $h_k$, let us plot the absolute errors, as we did for the $t = 0$ case. This time, we lack an analytical formula for the exact time propagation (remember that the initial wavepacket was the refrence for $t = 0$). Therefore, the best reference is obviously the propagated wavapacket obtained using the largest $k_{max}$ and the smallest $h_k$ value.

In [ ]:
# Choose the last time of time_grid
i_t = -1
t = time_grid[i_t]
# The reference is the propagated wavepacket obtained with 
# the highest k_max for the lowest h_k
i_b_ref = -1
ref = np.abs(exact_tps[i_b_ref][i_t])
# Loop to make one plot for each h_k
for i_h, h in enumerate(h_ks):
    for i_k, k in enumerate(k_maxs):
        lab = "h_k = {}, k_max = {}".format(h, k)
        i_b = i_h*len(k_maxs)+i_k
        tp = np.abs(exact_tps[i_b][i_t])
        if i_b != i_b_ref:
            plt.plot(xgrid, np.abs(tp-ref), label=lab)
    plt.yscale('log')
    plt.title("t = {}".format(t))
    plt.legend(loc=6, bbox_to_anchor=(1, 0.5))
    plt.show()

As suspected, the absolute errors are high (around $10^{-1}$) for the largest $h_k$ values, even for the largest $k_{max}$. Decreasing $h_k$ leads to lower absolute errors, and finally, when $h_k$ is small enough, the abolute error reaches a minimum, that cannot be decreased by reducing $h_k$. 

As you can see, the converged propagated wavepacket for $k_{max} = 5$ is obtained with for $h_k = 0.05$, and the absolute error is already below $10^{-2}$ (this low value is due to the RAGE theorem, stating that the continuum states contributions to the time propagation tend to 0 for long times; remember the absolute error was above $10^{-1}$ for $t = 0$). 

However, the converged propagated wavepacket for $k_{max} = 15$ is reached for $h_k = 0.025$. As a comparison, this convergence happened for $h_k \lesssim 0.5$ for $t = 0$. This means that the number of continuum states required to reach a convergence for time $t = 3$ is an order of magnitude higher, and this number increases with $t$. 

The reason for that is the exponential term in the integrand of the exact time propagation: 

$f_{exact}(x, t) = \sum_b \left\langle \varphi_b | f \right\rangle \varphi_b(x) e^{- i E_b t} + 
                   \sum_{p=\pm} \int_0^\infty \text{d} k \left\langle \varphi_p | f \right\rangle \varphi_p(x) e^{- i E(k) t}$

The integrand being more oscillatory as t increases, a finer wavenumber grid is therefore required. The Siegert states expansions avoids such an integration, therefore making it as simple to study the long and the short time limit, while reducing drastically the number of states to use. However, the correct expansiopn to use is still to be found, even though we saw in another notebook that there is a transition between the MLE expansion and the Berggren expansion. Also, this expanion is only valid in region $II$, inside the potential well.

Coming back to the exact time propagation, let us conclude that every time you increase the maximal value in the time grid for the propagation of a wavepacket, it is advised to make sure that the value of $h_k$ is low enough (while keeping the same $k_{max}$ you were happy with for the initial time $t = 0$). Still, remember that the continuum contributions to the time propagation tend to zero as the time increases: it might be computationally prohibitive to reduce $h_k$ at some point, while using the bound states only would be sufficient to get accurate results.

### Convergence with respect to $h_k$

This part of the notebook does not really give new messages, but it may help in giving a clearer view on the influence of $h_k$ on the exact time propagation.

To that end, the wavapacket for different times is plotted while varying $h_k$ and keeping $k_{max}$ fixed (here, it is chosen to be the highest one). 

In [ ]:
# Choose the highest k_max
i_k = -1
k = k_maxs[i_k]
# Loop over the time to plot the convergence as a function of h_k
for i_t, t in enumerate(time_grid):
    for i_h, h in enumerate(h_ks):
        lab = "h_k = {}, k_max = {}".format(h, k)
        if i_k == -1:
            i_k = len(k_maxs) - 1
        i_b = i_h*len(k_maxs) + i_k
        tp = np.abs(exact_tps[i_b][i_t])
        plt.plot(xgrid, tp, label=lab)
    plt.title("t = {}".format(t))
    plt.legend(loc=6, bbox_to_anchor=(1, 0.5))
    plt.show()

As was seen earlier, it requires a smallest $h_k$ to reach convergence as the time increases: while $h_k = 0.5$ is small enough to get converged results for $t=0$, it is not the case anymore for $t = 0.25$. It is even required to use $h_k = 0.05$ for $t = 3$.

#### Absolute errors

We could also look at the absolute errors. For each time, the reference is the propagated wavepacket obtained using the lowest $h_k$ for the given $k_{max}$.

In [ ]:
# Choose the highest k_max
i_k = -1
k = k_maxs[i_k]
# Loop over the time to plot the convergence as a function of h_k
for i_t, t in enumerate(time_grid):
    # Define the reference (the highest h_k for the given k_max)
    if i_k == -1:
        i_k = len(k_maxs) -1
    i_b_ref = (len(h_ks)-1) * len(k_maxs) + i_k
    ref = np.abs(exact_tps[i_b_ref][i_t])
    # Loop over h_k to plot a new line
    for i_h, h in enumerate(h_ks):
        lab = "h_k = {}, k_max = {}".format(h, k)
        i_b = i_h*len(k_maxs)+i_k
        tp = np.abs(exact_tps[i_b][i_t])
        # Do not plot the obvious difference between the reference 
        # and itself
        if i_b != i_b_ref:
            plt.plot(xgrid, np.abs(tp-ref), label=lab)
    plt.yscale('log')
    plt.title("t = {}".format(t))
    plt.legend(loc=6, bbox_to_anchor=(1, 0.5))
    plt.show()